In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers

In [2]:
full_players = pd.read_csv('PlayerDataFull.csv')

In [3]:
unq_positions = full_players['Best Position'].unique()
unq_positions.sort()

In [4]:
# load the models
models = {}
for i in range(len(unq_positions)):
    model = tf.keras.Sequential()
    model.add(layers.Dense(units=1, activation=None, use_bias=False))
    model.load_weights('./weights/' + unq_positions[i] + 'model')
    models[unq_positions[i]] = model

W0706 17:00:39.013906  3988 deprecation.py:506] From C:\Users\tas12\Anaconda3\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
# load full player data
player_data_full = {}
player_data_model = {}
for i in range(len(unq_positions)):
    player_data_full[unq_positions[i]] = pd.read_csv('./databyposition/' + unq_positions[i] + 'full.csv')
    player_data_model[unq_positions[i]] = pd.read_csv('./databyposition/' + unq_positions[i] + '.csv')

In [6]:
for k, v in player_data_full.items():
    print (k, v.shape[0])

CAM 1314
CB 2425
CDM 920
CF 151
CM 640
GK 1346
LB 534
LM 436
LW 307
LWB 200
RB 580
RM 825
RW 850
RWB 244
ST 1674


In [7]:
predictions = {}
for pos in unq_positions:
    predictions[pos] = models[pos].predict(player_data_model[pos].values)
for key in predictions:
    predictions[key] = predictions[key].flatten() - player_data_full[key]['Overall'].values

In [8]:
print (predictions)

{'CAM': array([-0.91768646, -0.94911957, -0.4259491 , ...,  0.06906891,
       -0.17984009, -0.86090088]), 'CB': array([ 0.36454773, -0.38365173, -0.35622406, ..., -1.1688118 ,
       -0.03526306, -0.01248932]), 'CDM': array([-8.17573547e-01, -3.51562500e-01,  3.88420105e-01, -1.57625580e+00,
       -5.47798157e-01, -3.26171875e-01, -8.36997986e-01, -2.86979675e-01,
       -3.01193237e-01,  2.07298279e-01, -5.37857056e-01, -2.79533386e-01,
       -8.54721069e-02, -8.07571411e-01,  1.68251801e+00, -3.68011475e-01,
       -8.74191284e-01, -2.06409454e+00, -1.23358154e+00,  1.88644409e+00,
       -5.76362610e-01,  5.27008057e-01,  2.07486725e+00, -5.79307556e-01,
       -9.29321289e-01, -8.53385925e-01, -5.44853210e-01, -6.63276672e-01,
       -5.71090698e-01, -1.84320831e+00,  1.54950714e+00, -1.77192688e-01,
        5.97198486e-01, -1.20543671e+00,  5.14907837e-02, -1.10207367e+00,
       -1.01318359e-01, -3.90396118e-02, -5.07461548e-01, -4.81338501e-01,
       -7.06985474e-01, -9.5397

In [9]:
diffs = {}
for key in predictions:
    vals = []
    vals.append(np.argmax(predictions[key]))
    vals.append(np.max(predictions[key]))
    vals.append(np.argmin(predictions[key]))
    vals.append(np.min(predictions[key]))
    diffs[key] = vals

In [10]:
for key in predictions:
    print (key)
    argmax, maxval, argmin, minval = diffs[key]
    playermax = player_data_full[key].iloc[argmax,:]['Name']
    playermin = player_data_full[key].iloc[argmin,:]['Name']
    print ('{} is one of the furthest away at {}'.format(playermax,maxval))
    print ('{} is the other of the furthest away at {}'.format(playermin, minval))

CAM
Gjelbrim Taipi is one of the furthest away at 3.003032684326172
Thomas Müller is the other of the furthest away at -3.3851165771484375
CB
Dong Jin Park is one of the furthest away at 6.159934997558594
Thiago Emiliano da Silva is the other of the furthest away at -2.5772171020507812
CDM
Fernando Saucedo is one of the furthest away at 4.519371032714844
Carlos Henrique Venancio Casimiro is the other of the furthest away at -2.0640945434570312
CF
Valon Fazliu is one of the furthest away at 2.924285888671875
Dele Alli is the other of the furthest away at -1.8776092529296875
CM
Matt Taylor is one of the furthest away at 5.63275146484375
Paul Pogba is the other of the furthest away at -2.5307235717773438
GK
Vicente Guaita Panadero is one of the furthest away at 0.6813735961914062
Thibaut Courtois is the other of the furthest away at -1.5267486572265625
LB
César Landa is one of the furthest away at 4.706027984619141
Marcel Halstenberg is the other of the furthest away at -1.65301513671875


In [11]:
# get the best (and worst!) of each position
preds_other_pos = {}
preds_bad_other_pos = {}

for pos in unq_positions:
    curr_preds = []
    curr_preds_bad = []
    for pos2 in unq_positions:
        preds = models[pos].predict(player_data_model[pos2].values).flatten()
        best = np.argmax(preds)
        worst = np.argmin(preds)
        player = player_data_full[pos2].iloc[best,:]['Name']
        player_worst = player_data_full[pos2].iloc[worst,:]['Name']
        curr_preds.append('{} is the best {} of the {}s'.format(player, pos, pos2))
        curr_preds_bad.append('{} is the worst {} of the {}s'.format(player_worst, pos, pos2))
    preds_other_pos[pos] = curr_preds
    preds_bad_other_pos[pos] = curr_preds_bad

In [12]:
count_players = {}
for pos in unq_positions:
    curr = preds_other_pos[pos]
    for player in curr:
        play = player.split(' is ')[0]
        if play not in count_players.keys():
            count_players[play] = [pos]
        else:
            count_players[play].append(pos)
for k, v in count_players.items():
    print (k, len(v),  v)

Lionel Messi 7 ['CAM', 'CF', 'LM', 'LW', 'RM', 'RW', 'ST']
Thomas Delaney 7 ['CAM', 'CM', 'LM', 'LW', 'RM', 'RW', 'ST']
Radja Nainggolan 6 ['CAM', 'CF', 'LW', 'RM', 'RW', 'ST']
Sergio Agüero 3 ['CAM', 'CF', 'ST']
Luka Modrić 6 ['CAM', 'CM', 'LM', 'LW', 'RM', 'RW']
Ederson Santana de Moraes 9 ['CAM', 'CDM', 'CF', 'LM', 'LW', 'LWB', 'RM', 'RW', 'ST']
David Alaba 6 ['CAM', 'CM', 'LM', 'LW', 'RM', 'RW']
Marcelo Vieira da Silva 13 ['CAM', 'CB', 'CDM', 'CF', 'CM', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM', 'RW', 'RWB']
Antoine Griezmann 10 ['CAM', 'CDM', 'CF', 'CM', 'LM', 'LW', 'RB', 'RM', 'RW', 'ST']
Jordi Alba Ramos 11 ['CAM', 'CF', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM', 'RW', 'RWB', 'ST']
Joshua Kimmich 7 ['CAM', 'CF', 'CM', 'LM', 'LW', 'RM', 'RW']
Ronaldo Jailson Cabrais Petri 5 ['CAM', 'LM', 'LW', 'RM', 'RW']
Neymar da Silva Santos Jr. 7 ['CAM', 'CF', 'CM', 'LM', 'LW', 'RM', 'RW']
Sergio Roberto Carnicer 3 ['CAM', 'CDM', 'CM']
C. Ronaldo dos Santos Aveiro 8 ['CAM', 'CF', 'CM', 'LM', 'LW', 'RM',

In [13]:
count_players = {}
for pos in unq_positions:
    curr = preds_bad_other_pos[pos]
    for player in curr:
        play = player.split(' is ')[0]
        if play not in count_players.keys():
            count_players[play] = [pos]
        else:
            count_players[play].append(pos)
for k, v in count_players.items():
    print (k, len(v),  v)

Julius Tauriainen 4 ['CAM', 'LM', 'RM', 'RW']
Kyle Watkins 4 ['CAM', 'CB', 'CF', 'ST']
Alessandro De Angelis 13 ['CAM', 'CB', 'CF', 'CM', 'LB', 'LM', 'LW', 'LWB', 'RB', 'RM', 'RW', 'RWB', 'ST']
Ben Worman 10 ['CAM', 'CDM', 'CF', 'CM', 'GK', 'LM', 'LW', 'RM', 'RW', 'ST']
Lee Watkins 6 ['CAM', 'CF', 'LM', 'LW', 'RM', 'RW']
Maarten Vandevoordt 5 ['CAM', 'LM', 'LW', 'RM', 'RW']
Declan Dunn 4 ['CAM', 'CDM', 'CF', 'CM']
Julian Conze 1 ['CAM']
Thais Damgaard Nielsen 11 ['CAM', 'CF', 'CM', 'LB', 'LM', 'LW', 'RB', 'RM', 'RW', 'RWB', 'ST']
Rhys Norrington-Davies 8 ['CAM', 'CDM', 'CF', 'CM', 'LM', 'LW', 'RM', 'RW']
Nicolás Fuentes 8 ['CAM', 'CDM', 'CF', 'CM', 'LM', 'LW', 'RM', 'RW']
Dean Ager 1 ['CAM']
Nazifi Yahaya 7 ['CAM', 'CM', 'LM', 'LW', 'LWB', 'RM', 'RW']
Xin Wang 10 ['CAM', 'CDM', 'CF', 'CM', 'LB', 'LM', 'RB', 'RM', 'RW', 'RWB']
Noah Christoffersson 11 ['CAM', 'CF', 'CM', 'LB', 'LM', 'LW', 'RB', 'RM', 'RW', 'RWB', 'ST']
Roberto López Alcaide 1 ['CB']
Dean Williams 5 ['CB', 'LB', 'LWB', 'R